# Tiền xử lí dữ liệu

## Giới thiệu

Notebook này bao gồm các bước tiền xử lý dữ liệu để chuẩn bị cho việc phân tích và xây dựng mô hình dự đoán cho dự án.

## Mục tiêu

- Đánh giá, xác định hướng xử lý cho các ô/ cột dữ liệu bị thiếu thông tin
- Điền các thông tin bị khuyết
- Feature enginering
- Gom các bảng dữ liệu 
- Chuẩn hóa dữ liệu cho mô hình
- Lọc/ xử lý outliner 

### Import thư viện cần thiết

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import missingno as msno

### Load datasets

In [ ]:
df_chotot = pd.read_csv("../datasets/raw_chotot_car_features.csv")
df_bonbanh = pd.read_csv("../datasets/raw_bonbanh_car_features.csv")

### I. Đánh giá dữ liệu thô
#### Chotot

In [ ]:
df_chotot.info()

- Dataset có 4928 samples, 21 features, tuy nhiên có nhiều cột đang sai định dạng
- Cần chuyển các cột về đúng định dạng `numerical/categorial`

In [ ]:
df_chotot.head()

- `title`: Thường chứa hãng/tên xe, năm sản xuất, dung tích, odo (km) - nhưng hầu hết phần này đã được tách ra vào các cột riêng trong dataset. Vì vậy giữ `title` nguyên bản để tham chiếu là đủ; không nên phụ thuộc vào `title` để tạo các trường chính cho mô hình.

- `price`: giá hiện lưu dưới dạng chuỗi với dấu phân nghìn `.` và hậu tố `đ` (ví dụ: `320.000.000 đ`, `123.879.846 đ`). Một số giá có dạng không tròn (thực tế VND), giá có khoảng rộng (từ vài chục triệu tới > 2 tỷ).
  - Xóa dấu `.` và ký tự không số, bỏ hậu tố `đ`, chuyển về số nguyên VND.
  - Kiểm tra ngoại lệ (giá = 0, NULL, hoặc giá quá nhỏ/lớn so với phân phối) và xử lý (loại bỏ hoặc gắn nhãn outlier).

- `Số Km đã đi`: phần lớn là số nguyên (ví dụ `88000`, `275000`), nhưng có vài giá trị bất thường (ví dụ `999999` trong dataset mẫu — có khả năng là placeholder hoặc lỗi). 
  - Chuyển về numeric, thay các giá trị rất lớn (>= 900000) thành `NaN` để đánh dấu outlier/placeholder.

- `Năm sản xuất`: có trong cột riêng; cần chuyển về `int` và kiểm tra phạm vi hợp lệ (ví dụ 1980 ≤ year ≤ current_year + 1). Giá trị ngoài khoảng này nên được gắn `NaN` để xử lý sau.

- `Hộp số`, `Nhiên liệu`, `Kiểu dáng`, `Số chỗ`: là các categorical có vocabulary rõ ràng nhưng không đồng nhất (ví dụ `Tự động` / `Số sàn` / `Bán tự động` / blank). Hành động:
  - Điền missing bằng cách gọi API của `https://www.auto-data.net`.

- `Số đời chủ`: hiện lưu dạng text (`1 chủ`, `> 1 chủ`, hoặc blank). Map sang numeric: `1 chủ` → 1, `> 1 chủ` → 2, blank → `NaN`.

- `Có phụ kiện đi kèm`: giá trị `Có`/`Không`/blank → map thành boolean (True/False) với blank → `NaN` hoặc False (cho rằng không đưa thông tin là không có).

- `location`: thường chứa phường/quận và tỉnh/thành — thông tin có thể rút thành `city`. Tuy nhiên format không đồng nhất; nên parse bằng rule đơn giản (tách theo dấu phẩy và lấy phần cuối làm `city`) và kiểm tra tỷ lệ thành công.

In [ ]:
msno.matrix(df_chotot)

In [ ]:
def get_numerical_summary(df):
    total = df.shape[0]
    missing_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
    missing_percent = {}
    for col in missing_columns:
        null_count = df[col].isnull().sum()
        per = (null_count/total) * 100
        missing_percent[col] = per
        print("{} : {} ({}%)".format(col, null_count, round(per, 3)))
    return missing_percent

In [ ]:
chotot_missing_summary = get_numerical_summary(df_chotot)

- `Số Km đã đi` — 213 missing (4.322%): tỷ lệ thấp; xử lý đơn giản bằng cách impute median hoặc mean, hoặc giữ NaN và dùng indicator `km_missing`. Những giá trị placeholder lớn (ví dụ 999999) nên được convert thành NaN trước khi impute.

- `Còn hạn đăng kiểm` — 2872 missing (58.279%): thiếu nhiều; cột này không ổn định để dùng trực tiếp. 

- `Kiểu dáng` — 840 missing (17.045%): tỷ lệ vừa phải — nên cố gắng chuẩn hóa và điền bằng mode hoặc map theo `title` nếu khả thi; nếu không, fill = 'unknown'.

- `Số chỗ` — 548 missing (11.12%): mức vừa phải — impute bằng mode theo `Kiểu dáng`/`Hãng` hoặc fill `'unknown'`.

- `Trọng lượng`, `Trọng tải` — 213 missing mỗi cột (4.322%): tỷ lệ thấp; impute bằng mode hoặc giữ NaN nếu ít quan trọng.

- `location` — 149 missing (3.024%): rất thấp; cố gắng parse `city` hoặc sử dụng model để fill phần còn lại.

- `seller` — 4928 missing (100.0%): hoàn toàn trống trong mẫu — không dùng cột này.

- `Số đời chủ` — 2808 missing (56.981%): thiếu nhiều; cột này không ổn định để dùng trực tiếp. 

- `Có phụ kiện đi kèm` — 3332 missing (67.614%): thiếu nhiều; cột này không ổn định để dùng trực tiếp.

#### BonBanh